In [1]:
%load_ext autoreload
%autoreload 2
from keras.layers import Dense, maximum
from .layers import repeat_layers
from .permutational_layer import *

Using TensorFlow backend.


In [2]:
pairwise_model = PairwiseModel((4,), repeat_layers(Dense, [32, 8], name='hidden'), name="pairwise_model")
pairwise_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x1 (InputLayer)                 (None, 4)            0                                            
__________________________________________________________________________________________________
x2 (InputLayer)                 (None, 4)            0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 8)            0           x1[0][0]                         
                                                                 x2[0][0]                         
__________________________________________________________________________________________________
hidden_1 (Dense)                (None, 32)           288         concatenate_1[0][0]              
__________

In [3]:
perm_encoder = PermutationalEncoder(pairwise_model, 3 ,name='permutational_encoder')
perm_encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x1_main (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
x2 (InputLayer)                 (None, 4)            0                                            
__________________________________________________________________________________________________
x3 (InputLayer)                 (None, 4)            0                                            
__________________________________________________________________________________________________
pairwise_model (Model)          (None, 8)            552         x1_main[0][0]                    
                                                                 x1_main[0][0]                    
          

In [4]:
perm_layer = PermutationalLayer(perm_encoder, name='permutational_layer')
perm_layer.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x1 (InputLayer)                 (None, 4)            0                                            
__________________________________________________________________________________________________
x2 (InputLayer)                 (None, 4)            0                                            
__________________________________________________________________________________________________
x3 (InputLayer)                 (None, 4)            0                                            
__________________________________________________________________________________________________
permutational_encoder (Model)   (None, 8)            552         x1[0][0]                         
                                                                 x2[0][0]                         
          

In [5]:
x = [[[1,2,3,4]],
     [[4,3,2,1]],
     [[1,2,3,4]]]
predictions = perm_layer.predict(x)
print('# Input Data')
print(x)
print('# Output Features')
for pred in predictions:
    print(pred)
print("You will notice that output1 = output2 if input1 = input2. So the permutational layer is simply performing a function f(x) for each input x.")
print("It is similar to a shared model applied to each input.")
print("The difference that makes this approach better is because f(x) is not standalone like shared model. It sees useful relationship between inputs.")

# Input Data
[[[1, 2, 3, 4]], [[4, 3, 2, 1]], [[1, 2, 3, 4]]]
# Output Features
[[-1.0746211  -0.9102764  -2.225684    1.944071    3.3657804   2.1630723
   0.00699236  0.83873504]]
[[-3.6580932 -2.3145645 -1.2344245  2.7202842  1.9061317  4.5679073
   1.383071   2.9916458]]
[[-1.0746211  -0.9102764  -2.225684    1.9440708   3.3657804   2.163072
   0.00699236  0.83873504]]
You will notice that output1 = output2 if input1 = input2. So the permutational layer is simply performing a function f(x) for each input x.
It is similar to a shared model applied to each input.
The difference that makes this approach better is because f(x) is not standalone like shared model. It sees useful relationship between inputs.


In [6]:
perm_layer2 = PermutationalLayer(PermutationalEncoder(PairwiseModel((8,), repeat_layers(Dense, [16, 4], activation='relu')), 3), name='permutational_layer2')
perm_layer3 = PermutationalLayer(PermutationalEncoder(PairwiseModel((4,), repeat_layers(Dense, [16, 4], activation='tanh')), 3), pooling=maximum, name='permutational_layer3')

In [7]:
# now let's make a model with multi permutational layers to show you that it's possible!
a = Input((4,), name='a')
b = Input((4,), name='b')
c = Input((4,), name='c')
inputs = [a, b, c]
outputs = perm_layer(inputs)
outputs = perm_layer2(outputs)
output = perm_layer3(outputs) # this layer applies a pooling function to the outputs list
model = Model(inputs, output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
a (InputLayer)                  (None, 4)            0                                            
__________________________________________________________________________________________________
b (InputLayer)                  (None, 4)            0                                            
__________________________________________________________________________________________________
c (InputLayer)                  (None, 4)            0                                            
__________________________________________________________________________________________________
permutational_layer (Model)     [(None, 8), (None, 8 552         a[0][0]                          
                                                                 b[0][0]                          
          

In [8]:
# let's predict with the big mult layers model on some similar data
print("# Multi-layer Prediction")
print("Because the layer is preserving permutation invariance. The output will be the same no matter how many times you permute the input order.")
print('## Input/Output 1')
x = [[[0,0,0,0]],
     [[1,2,3,4]],
     [[4,3,2,1]]]
print(x)
print(model.predict(x))

print('## Input/Output 2')
x = [[[1,2,3,4]],
     [[0,0,0,0]],
     [[4,3,2,1]]]
print(x)
print(model.predict(x))

print('## Input/Output 3')
x = [[[4,3,2,1]],
     [[0,0,0,0]],
     [[1,2,3,4]]]
print(x)
print(model.predict(x))
print()
print("Isn't this cool !?")

# Multi-layer Prediction
Because the layer is preserving permutation invariance. The output will be the same no matter how many times you permute the input order.
## Input/Output 1
[[[0, 0, 0, 0]], [[1, 2, 3, 4]], [[4, 3, 2, 1]]]
[[ 0.25130957  0.02237612  0.01398425 -0.01015728]]
## Input/Output 2
[[[1, 2, 3, 4]], [[0, 0, 0, 0]], [[4, 3, 2, 1]]]
[[ 0.25130957  0.02237625  0.01398422 -0.0101573 ]]
## Input/Output 3
[[[4, 3, 2, 1]], [[0, 0, 0, 0]], [[1, 2, 3, 4]]]
[[ 0.2513096   0.02237615  0.01398424 -0.01015728]]

Isn't this cool !?
